In [9]:
!pip install spacy-transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 4.4 MB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [11]:
!git clone https://github.com/tulee3474/AI_Resume_Builder

Cloning into 'AI_Resume_Builder'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 370 (delta 17), reused 104 (delta 12), pack-reused 232
Receiving objects: 100% (370/370), 2.04 MiB | 2.85 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [13]:
%cd /content/AI_Resume_Builder/
!git pull

/content/AI_Resume_Builder
Already up to date.


In [14]:
import json
cv_data=json.load(open('/content/AI_Resume_Builder/Resume_annotated/merged res.json','r'))

In [15]:
!python -m spacy init fill-config /content/AI_Resume_Builder/base_config.cfg /content/AI_Resume_Builder/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/AI_Resume_Builder/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [16]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [17]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [18]:
len(train), len(test)

(69, 30)

In [19]:
cv_data[0]

{'id': 297,
 'label': [[0, 13, 'ability'],
  [623, 655, 'work experience'],
  [671, 724, 'work experience'],
  [1202, 1249, 'work experience'],
  [1274, 1349, 'work experience'],
  [1399, 1437, 'work experience'],
  [1600, 1655, 'award'],
  [1686, 1712, 'major'],
  [1713, 1716, 'degree'],
  [1725, 1757, 'college'],
  [1900, 1921, 'education'],
  [2020, 2026, 'skill'],
  [2027, 2034, 'skill'],
  [2035, 2041, 'skill'],
  [2058, 2074, 'skill'],
  [2075, 2095, 'skill'],
  [2097, 2100, 'skill'],
  [2101, 2106, 'skill'],
  [2108, 2112, 'skill'],
  [2113, 2123, 'skill'],
  [2125, 2129, 'skill'],
  [2130, 2133, 'skill'],
  [2135, 2138, 'skill'],
  [2165, 2182, 'ability'],
  [2183, 2198, 'ability'],
  [2199, 2215, 'ability'],
  [2216, 2229, 'ability'],
  [2336, 2363, 'skill'],
  [2402, 2420, 'skill'],
  [2456, 2477, 'skill']],
 'text': 'Web Developer Resume Template (Full Text Version)\nOlivier Baudet\nAddress: 44 place de Miremont, 77170, Paris, France\nEmail address: olivierbaudet@gmail.com\n

In [20]:
file=open('error.txt','w')
db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [21]:
db.tokens

[array([[ 3810947486225571421,  5838559663594637404,                    0,
         ...,                    0,                    0,
                            0],
        [12322875177300609251,  9562076534475284812,                    0,
         ...,                    0,                    0,
                            0],
        [  962983613142996970,   962983613142996970,                    0,
         ...,                    0,                    0,
                            0],
        ...,
        [ 4287584610774053875,   790251972160944543,                    0,
         ...,                    0,                    0,
                            0],
        [ 2876290348908956490,  3754547387801542680,                    0,
         ...,                    0,                    0,
                            0],
        [12646065887601541794, 12646065887601541794,                    0,
         ...,                    0,                    0,
                            0

In [22]:
!python -m spacy train /content/AI_Resume_Builder/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-08 02:25:32,439] [INFO] Set up nlp object from config
[2022-08-08 02:25:32,449] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-08 02:25:32,453] [INFO] Created vocabulary
[2022-08-08 02:25:32,454] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 487kB/s]
Downloading: 100% 878k/878k [00:01<00:00, 811kB/s] 
Downloading: 100% 446k/446k [00:00<00:00, 497kB/s]
Downloading: 100% 1.29M/1.29M [00:01<00:00, 1.22MB/s]
Downloading: 100% 478M/478M [00:12<00:00, 40.9MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a mode

In [23]:
nlp=spacy.load('/content/AI_Resume_Builder/output/model-best') 

In [24]:
doc=nlp('Seasoned and independent Front End Developer with 5 years of experience in blending the art of design with skill of programming to deliver an immersive and engaging user experience through efficient website development, proactive feature optimization, and relentless debugging.')
for ent in doc.ents:
  print(ent.text, "    ->>>> ", ent.label_)

In [25]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import sys, fitz

In [27]:
for x in range(1,11):
  fname='/content/AI_Resume_Builder/res test/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('resume num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

resume num 1 test
ASP.NET  ->>>>>>  skill
C  ->>>>>>  skill
• Operating system:-  ->>>>>>  work experience
• Language Known:-  ->>>>>>  work experience
C  ->>>>>>  skill
• Concepts Known:-  ->>>>>>  work experience
DBMS  ->>>>>>  skill
Computer architecture  ->>>>>>  skill
DSA  ->>>>>>  skill
• Front End  ->>>>>>  work experience
HTML  ->>>>>>  skill
CSS  ->>>>>>  skill
XML  ->>>>>>  skill
AJAX  ->>>>>>  skill
• Web Development:-  ->>>>>>  work experience
ASP.NET  ->>>>>>  skill
• RDBMS:- SQL  ->>>>>>  work experience


resume num 2 test
SQL  ->>>>>>  skill
LINQ  ->>>>>>  skill
Entity Frame work  ->>>>>>  skill
HTML5  ->>>>>>  skill
Jquery  ->>>>>>  skill
AJAX  ->>>>>>  skill
MVC  ->>>>>>  skill
WCF  ->>>>>>  skill
Web Services  ->>>>>>  skill
Software Developer  ->>>>>>  work experience
.Net Developer  ->>>>>>  work experience
Information Technology  ->>>>>>  major
.Net  ->>>>>>  skill
Jquery  ->>>>>>  skill
Mvc  ->>>>>>  skill
Linq  ->>>>>>  skill
Entity framework  ->>>>>>  skill
Web